In [1]:
# Load all the CSV

In [2]:
!git clone https://github.com/pthengtr/kcw-analytics.git

Cloning into 'kcw-analytics'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 136 (delta 67), reused 71 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (136/136), 114.30 KiB | 4.76 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [3]:
!cd /content/kcw-analytics && git pull origin main

From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
Already up to date.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")



Loaded: raw_hq_pimas_purchase_bills.csv -> (82716, 49)
Loaded: raw_hq_simas_sales_bills.csv -> (481937, 49)
Loaded: raw_hq_pidet_purchase_lines.csv -> (246580, 41)
Loaded: raw_hq_sidet_sales_lines.csv -> (1187205, 38)
Loaded: raw_hq_icmas_products.csv -> (114755, 94)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2721, 49)
Loaded: raw_syp_simas_sales_bills.csv -> (10368, 49)
Loaded: raw_syp_sidet_sales_lines.csv -> (30374, 38)
Loaded: raw_syp_pidet_purchase_lines.csv -> (25555, 41)
Loaded: raw_inventory_hq_2024.csv -> (4983, 8)


In [6]:
import sys
import importlib

# ensure repo is on path
repo_path = "/content/kcw-analytics"
if repo_path not in sys.path:
    sys.path.append(repo_path)

# import the module (NOT individual functions)
import src.kcw.utils as utils

# reload to pick up latest .py changes
importlib.reload(utils)


<module 'src.kcw.utils' from '/content/kcw-analytics/src/kcw/utils.py'>

In [7]:
get_vat_sales_lines = utils.get_vat_sales_lines
get_vat_purchase_lines = utils.get_vat_purchase_lines
get_vat_sales_lines_last_purchase_nonvat = utils.get_vat_sales_lines_last_purchase_nonvat
get_nonvat_sales_lines_last_purchase_vat = utils.get_nonvat_sales_lines_last_purchase_vat
get_hq_to_syp_transfer_lines = utils.get_hq_to_syp_transfer_lines
get_syp_received_transfer_lines = utils.get_syp_received_transfer_lines
build_inventory_summary_avg_cost = utils.build_inventory_summary_avg_cost
get_ap_unpaid_bills = utils.get_ap_unpaid_bills
get_ar_unpaid_bills = utils.get_ar_unpaid_bills
build_yearly_inventory_report = utils.build_yearly_inventory_report


# **Test Function**

In [8]:
YEAR = 2025
PREV_YEAR = YEAR - 1

vat_sales_hq = get_vat_sales_lines(data, year=YEAR, source="hq")
vat_sales_syp = get_vat_sales_lines(data, year=YEAR, source="syp")

vat_purchase_hq = get_vat_purchase_lines(data, year=YEAR, source="hq")
vat_purchase_syp = get_vat_purchase_lines(data, year=YEAR, source="syp")

vat_sales_last_purchase_nonvat_hq = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="hq"
)
vat_sales_last_purchase_nonvat_syp = get_vat_sales_lines_last_purchase_nonvat(
    data, year=YEAR, source="syp"
)

nonvat_sales_last_purchase_vat_hq = get_nonvat_sales_lines_last_purchase_vat(
    data, year=YEAR, source="hq"
)
nonvat_sales_last_purchase_vat_syp = get_nonvat_sales_lines_last_purchase_vat(
    data, year=YEAR, source="syp"
)

hq_tfv_lines = get_hq_to_syp_transfer_lines(data, year=YEAR)
syp_tfv_lines = get_syp_received_transfer_lines(data, year=YEAR)


TF rows count: (9752, 38)
Unique TF bills: 1150
TF rows count: (9750, 41)
Unique TF bills: 1148


In [9]:
inv_hq = build_inventory_summary_avg_cost(
    in_dfs=[
        vat_purchase_hq,
        vat_sales_last_purchase_nonvat_hq,
    ],
    out_dfs=[
        vat_sales_hq,
        hq_tfv_lines,
    ],
    pidet_all_df=data["raw_hq_pidet_purchase_lines.csv"],  # ALL TIME
)

In [10]:
inv_syp = build_inventory_summary_avg_cost(
    in_dfs=[
        syp_tfv_lines,
        vat_sales_last_purchase_nonvat_syp,
    ],
    out_dfs=[
        vat_sales_syp,
    ],
    pidet_all_df=data["raw_hq_pidet_purchase_lines.csv"],  # ALL TIME
)

In [11]:
ap_hq  = get_ap_unpaid_bills(data, year=YEAR, site="hq")
ap_syp = get_ap_unpaid_bills(data, year=YEAR, site="syp")

ar_hq  = get_ar_unpaid_bills(data, year=YEAR, site="hq")
ar_syp = get_ar_unpaid_bills(data, year=YEAR, site="syp")

[AP] site=hq year=2025 source=raw_hq_pimas_purchase_bills.csv
Rows: (2261, 51) | Unique BILLNO: 2261
Check (PAID=Y & missing VOUCDATE2) rows: 0
[AP] site=syp year=2025 source=raw_syp_pimas_purchase_bills.csv
Rows: (1, 51) | Unique BILLNO: 1
Check (PAID=Y & missing VOUCDATE2) rows: 0
[AR] site=hq year=2025 source=raw_hq_simas_sales_bills.csv
Rows: (3983, 51) | Unique BILLNO: 3982
Check (PAID=Y & missing VOUCDATE2) rows: 0
[AR] site=syp year=2025 source=raw_syp_simas_sales_bills.csv
Rows: (1125, 51) | Unique BILLNO: 1125
Check (PAID=Y & missing VOUCDATE2) rows: 0


In [12]:
import pandas as pd

# read CSV (force no dtype guessing issues)
df = data[f"raw_inventory_hq_{PREV_YEAR}.csv"].copy()

# robust numeric conversion (Excel-like)
amt = (
    df["AMOUNT"]
    .astype(str)
    .str.replace(",", "", regex=False)            # 1,234.56 -> 1234.56
    .str.replace(r"^\((.*)\)$", r"-\1", regex=True)  # (123.45) -> -123.45
    .str.strip()
)

df["AMOUNT"] = pd.to_numeric(amt, errors="coerce")

# Recompute AV_COST safely
df["AV_COST"] = (
    df["AMOUNT"] / df["END"]
).where(df["END"] != 0, 0)

# drop rows with missing cost
df_valid = df.dropna(subset=["AMOUNT"]).copy()

# results
print("Total rows:", len(df))
print("Rows with valid AMOUNT:", len(df_valid))
print("Dropped rows (missing cost):", len(df) - len(df_valid))
print("Final inventory AMOUNT total:", df_valid["AMOUNT"].sum())

Total rows: 4983
Rows with valid AMOUNT: 4978
Dropped rows (missing cost): 5
Final inventory AMOUNT total: 11256868.851814598


In [13]:
inv_hq

,BCODE,DESCR,IN,OUT,AV_COST
0,01010044,ซีลข้อเหวี่ยง IS TX74 TORA,4.0,2.0,210.373833
1,01010080,ซีลล้อหน้า TX NOK,8.0,8.0,81.150000
2,01010170,สกรูล้อหลัง สีเทา TX-S.68 RH LOCO,0.0,3.0,125.000000
3,01010268,แผงแอร์ FORWARD GIGA FR PACO,1.0,0.0,1450.000000
4,01010440,ซีลล้อหลัง ใน TX63 NOK,2.0,0.0,76.750000
...,...,...,...,...,...
11877,91010006,กางเกง,51.0,0.0,0.100000
11878,91010007,ร่ม,197.0,1.0,0.134615
11879,91010010,เสื้อคอปก,13.0,0.0,0.000000
11880,91010013,หมวกคุมหน้า สีเทา MAMMO,10.0,0.0,0.000000


In [14]:
inv_summary_hq = build_yearly_inventory_report(
    prev_year_inventory=df_valid,
    current_year_movement=inv_hq
)

inv_summary_syp = build_yearly_inventory_report(
    prev_year_inventory=None,
    current_year_movement=inv_syp
)

In [15]:
total_inventory_amount = inv_summary_hq["AMOUNT"].sum()
neg_count = inv_summary_hq["NEG_END_FIXED"].sum()

print("NEG_END_FIXED rows:", neg_count)
print("Total inventory AMOUNT:", total_inventory_amount)


NEG_END_FIXED rows: 988
Total inventory AMOUNT: 20957419.46236792


In [16]:
total_inventory_amount = inv_summary_syp["AMOUNT"].sum()
neg_count = inv_summary_syp["NEG_END_FIXED"].sum()

print("NEG_END_FIXED rows:", neg_count)
print("Total inventory AMOUNT:", total_inventory_amount)

NEG_END_FIXED rows: 26
Total inventory AMOUNT: 4096043.4845023514


In [17]:
inv_summary_hq

,BCODE,DESCR,BEGIN,BEGIN_AV_COST,BEGIN_AMOUNT,IN,OUT,END,AV_COST,AMOUNT,IN_ORIG,NEG_END_FIXED,IS_NEW_BCODE
0,01010044,ซีลข้อเหวี่ยง IS TX74 TORA,0.0,0.0,0.0,4.0,2.0,2.0,210.373833,420.747667,4.0,False,True
1,01010080,ซีลล้อหน้า TX NOK,0.0,0.0,0.0,8.0,8.0,0.0,81.150000,0.000000,8.0,False,True
2,01010170,สกรูล้อหลัง สีเทา TX-S.68 RH LOCO,0.0,0.0,0.0,3.0,3.0,0.0,125.000000,0.000000,0.0,True,True
3,01010268,แผงแอร์ FORWARD GIGA FR PACO,0.0,0.0,0.0,1.0,0.0,1.0,1450.000000,1450.000000,1.0,False,True
4,01010440,ซีลล้อหลัง ใน TX63 NOK,0.0,0.0,0.0,2.0,0.0,2.0,76.750000,153.500000,2.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13113,91010006,กางเกง,0.0,0.0,0.0,51.0,0.0,51.0,0.100000,5.100000,51.0,False,True
13114,91010007,ร่ม,0.0,0.0,0.0,197.0,1.0,196.0,0.134615,26.384615,197.0,False,True
13115,91010010,เสื้อคอปก,0.0,0.0,0.0,13.0,0.0,13.0,0.000000,0.000000,13.0,False,True
13116,91010013,หมวกคุมหน้า สีเทา MAMMO,0.0,0.0,0.0,10.0,0.0,10.0,0.000000,0.000000,10.0,False,True


In [18]:
inv_summary_syp

,BCODE,DESCR,BEGIN,BEGIN_AV_COST,BEGIN_AMOUNT,IN,OUT,END,AV_COST,AMOUNT,IN_ORIG,NEG_END_FIXED,IS_NEW_BCODE
0,01010044,ซีลข้อเหวี่ยง IS TX74 TORA,0,0.0,0.0,1.0,0.0,1.0,210.373833,210.373833,1.0,False,True
1,01010080,ซีลล้อหน้า TX NOK,0,0.0,0.0,2.0,0.0,2.0,81.15,162.300000,2.0,False,True
2,01010170,สกรูล้อหลัง สีเทา TX-S.68 RH LOCO,0,0.0,0.0,3.0,0.0,3.0,125.0,375.000000,3.0,False,True
3,01010830,ที่กดฟิตปั้มรุ่นใหม่,0,0.0,0.0,1.0,1.0,0.0,304.478252,0.000000,1.0,False,True
4,01011980,สวิทกุญแจสตาร์ท TX 12V/24V. EPINA,0,0.0,0.0,12.0,11.0,1.0,130.772727,130.772727,12.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,35050058,ซีลล้อหน้า-หลังยันมาร์ EF312-352T-393T CRR,0,0.0,0.0,6.0,5.0,1.0,75.597483,75.597483,6.0,False,True
5978,35050080,ชุดปากกระบอกใบมีด 45 มิล EF453T [KHL] 4 MCP,0,0.0,0.0,1.0,0.0,1.0,140.0,140.000000,1.0,False,True
5979,35050135,ซีลแกนเพลาไฮ/ซีลแกนคันชัก พมล. น้ำเ 35-45-10 ...,0,0.0,0.0,1.0,0.0,1.0,150.0,150.000000,1.0,False,True
5980,35050153,"ซีล PTO ยันมาร์ 2ชั้น 352-494 ,J/D531 MCP",0,0.0,0.0,2.0,1.0,1.0,125.833333,125.833333,2.0,False,True


In [19]:
total_sales_amount_hq = vat_sales_hq["AMOUNT"].sum()
total_sales_amount_syp = vat_sales_syp["AMOUNT"].sum()

print("Total sales hq:", total_sales_amount_hq)
print("Total sales syp:", total_sales_amount_syp)

Total sales hq: 49174154.31999999
Total sales syp: 4128908.7600000002


In [20]:
inv_summary_syp.to_csv(f"/content/drive/MyDrive/kcw_analytics/04_outputs/inv_summary_syp_{YEAR}.csv", index=False, encoding="utf-8-sig")
inv_summary_hq.to_csv(f"/content/drive/MyDrive/kcw_analytics/04_outputs/inv_summary_hq_{YEAR}.csv", index=False, encoding="utf-8-sig")

In [21]:
inv_summary_hq["AV_COST"].isna().sum()

np.int64(52)

In [22]:
inv_summary_syp["AV_COST"].isna().sum()

np.int64(10)

In [23]:
inv_summary_hq[inv_summary_hq["AV_COST"].isna()].head(10)

,BCODE,DESCR,BEGIN,BEGIN_AV_COST,BEGIN_AMOUNT,IN,OUT,END,AV_COST,AMOUNT,IN_ORIG,NEG_END_FIXED,IS_NEW_BCODE
774,03052175,เซนเซอร์ไมล์ D-MAX แท้,0.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,True,True
1950,05057622,"สวิทไฟขาเบรค ไร้สาย BIG-M,E24 ฟอนเท แท้",0.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,True,True
2164,07051666,"แป๊ปราวน้ำ นิวเรนเจอร์,เอเ แท้",0.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,True,True
2171,07051709,"ปะเก็นเฟืองท้าย เรนเจอร์ , ไฟเต",0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,False,True
2682,08050676,ซีลเพลาขับหน้า ABS TG 4WD LN165-16 แท้,0.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,True,True
2683,08050677,ซีลเพลาขับหน้า ไม่ABS TG 4WD LN165-16 แท้,0.0,0.0,0.0,1.0,1.0,0.0,NaN,0.0,0.0,True,True
2720,08051815,ใบพัดลม 4WD 7ใบ TG VIGO D4D 1-2KD A,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,False,True
2812,08054589,"กระบอกเบรคหลัง VIGO 2WD 13/16"" แท้พิเศษ",0.0,0.0,0.0,2.0,2.0,0.0,NaN,NaN,2.0,False,True
2829,08054774,หม้อลมเบรค 10นิ้ว VIGO ABS 1ชั่น JBS,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,False,True
2915,08055234,ซีลปั้มน้ำ 30 มิล T/T LN80,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,1.0,False,True


In [24]:
inv_summary_syp[inv_summary_syp["AV_COST"].isna()].head(10)

,BCODE,DESCR,BEGIN,BEGIN_AV_COST,BEGIN_AMOUNT,IN,OUT,END,AV_COST,AMOUNT,IN_ORIG,NEG_END_FIXED,IS_NEW_BCODE
444,03052175,เซนเซอร์ไมล์ D-MAX แท้,0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
1061,05057622,"สวิทไฟขาเบรค ไร้สาย BIG-M,E24 ฟอนเท แท้",0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
1075,06050435,สกรูล้อหลัง,0,0.0,0.0,6.0,6.0,0.0,<NA>,NaN,6.0,False,True
1171,07051666,"แป๊ปราวน้ำ นิวเรนเจอร์,เอเ แท้",0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
1403,08037840,ยางแท่นเกียร์ธรรมดา M VIGO ไม่เว้า RBI,0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
1423,08050676,ซีลเพลาขับหน้า ABS TG 4WD LN165-16 แท้,0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
1424,08050677,ซีลเพลาขับหน้า ไม่ABS TG 4WD LN165-16 แท้,0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
2272,11057072,ปะเก็นฝาสูบไฟ 4M40 ไททั่ล CNG Victor,0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
2300,11061531,ปะเก็นฝาสูบ 4D56 ไทรทั่น Cherry,0,0.0,0.0,1.0,0.0,1.0,<NA>,NaN,1.0,False,True
2750,13010345,สายลม หนา..ดูด/ดัน 3.5 หุน ดำคาดเห,0,0.0,0.0,11.6,1.6,10.0,<NA>,NaN,11.6,False,True


In [25]:
total_inventory_amount = inv_summary_hq["AMOUNT"].sum()
print("Total inventory AMOUNT:", total_inventory_amount)

Total inventory AMOUNT: 20957419.46236792


In [26]:
total_inventory_amount = inv_summary_hq["BEGIN_AMOUNT"].sum()
print("Total inventory BEGIN_AMOUNT:", total_inventory_amount)

Total inventory BEGIN_AMOUNT: 11256868.8518146
